# Fine-tunning SETR PUP on Parihaka Dataset

## Imports

In [1]:
from common import get_data_module, get_trainer_pipeline
from minerva.models.nets.image.setr import SETR_PUP
from torchmetrics import JaccardIndex

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Variaveis

In [2]:
root_data_dir = "/workspaces/HIAAC-KR-Dev-Container/shared_data/seam_ai_datasets/seam_ai/images"
root_annotation_dir = "/workspaces/HIAAC-KR-Dev-Container/shared_data/seam_ai_datasets/seam_ai/annotations"
ckpt_file = "/workspaces/HIAAC-KR-Dev-Container/shared_data/notebooks_e_pesos/SETR_pesos/vit_large_p16_new.pth"


img_size = (1008, 784)          # Change this to the size of the images in the dataset
model_name = "setr_pup"       # Model name (just identifier)
dataset_name = "seam_ai"        # Dataset name (just identifier)
single_channel = False          # If True, the model will be trained with single channel images (instead of 3 channels)

log_dir = "/workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/docs/notebooks/examples/seismic/facies_classification/parihaka/logs"              # Directory to save logs
batch_size = 1                  # Batch size    
seed = 42                       # Seed for reproducibility
num_epochs = 100                # Number of epochs to train
is_debug = True                 # If True, only 3 batch will be processed for 3 epochs
accelerator = "gpu"             # CPU or GPU
devices = 1                     # Num GPUs

## Data Module

In [3]:
data_module = get_data_module(
    root_data_dir=root_data_dir,
    root_annotation_dir=root_annotation_dir,
    img_size=img_size,
    batch_size=batch_size,
    seed=seed,
    single_channel=single_channel
)

data_module

DataModule
    Data: /workspaces/HIAAC-KR-Dev-Container/shared_data/seam_ai_datasets/seam_ai/images
    Annotations: /workspaces/HIAAC-KR-Dev-Container/shared_data/seam_ai_datasets/seam_ai/annotations
    Batch size: 1

In [4]:
# Just to check if the data module is working
data_module.setup("fit")
train_batch_x, train_batch_y = next(iter(data_module.train_dataloader()))
train_batch_x.shape, train_batch_y.shape

(torch.Size([1, 3, 1008, 784]), torch.Size([1, 1, 1008, 784]))

## **** Create and Load model HERE ****

In [5]:
model = SETR_PUP(
    image_size=img_size,
    num_classes=6,
    load_backbone_path=ckpt_file,
    original_resolution=(256, 576),
    train_metrics={"IoU": JaccardIndex(task="multiclass", num_classes=6)},
    val_metrics={"IoU": JaccardIndex(task="multiclass", num_classes=6)},
    test_metrics={"IoU": JaccardIndex(task="multiclass", num_classes=6)},
)


model

/workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/minerva/models/nets/image/setr.py:584: UserWarning: aux_output_layers not provided. Using default values [9, 14, 19].
  warnings.warn(
/workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/minerva/models/nets/image/setr.py:589: UserWarning: aux_weights not provided. Using default values [0.3, 0.3, 0.3].
  warnings.warn(


Interpolating positional embeddings to match the new image size.


SETR_PUP(
  (loss_fn): CrossEntropyLoss()
  (model): _SetR_PUP(
    (encoder): _VisionTransformerBackbone(
      (conv_proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
      (encoder): _Encoder(
        (dropout): Dropout(p=0.1, inplace=False)
        (layers): Sequential(
          (encoder_layer_0): EncoderBlock(
            (ln_1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (self_attention): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
            )
            (dropout): Dropout(p=0.1, inplace=False)
            (ln_2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (mlp): MLPBlock(
              (0): Linear(in_features=1024, out_features=4096, bias=True)
              (1): GELU(approximate='none')
              (2): Dropout(p=0.1, inplace=False)
              (3): Linear(in_features=4096, out_features=1024, bias=True)
              (4): Dropout(

## Pipeline

In [6]:
pipeline = get_trainer_pipeline(
    model=model,
    model_name=model_name,
    dataset_name=dataset_name,
    log_dir=log_dir,
    num_epochs=num_epochs,
    accelerator=accelerator,
    devices=devices,
    is_debug=is_debug,
    seed=seed,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 42


Log directory set to: /workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/docs/notebooks/examples/seismic/facies_classification/parihaka/logs/setr_pup/seam_ai


In [7]:
pipeline.run(data_module, task="fit")

/usr/local/lib/python3.10/dist-packages/lightning/fabric/loggers/csv_logs.py:268: Experiment logs directory /workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/docs/notebooks/examples/seismic/facies_classification/parihaka/logs/setr_pup/seam_ai exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!


Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/docs/notebooks/examples/seismic/facies_classification/parihaka/logs/setr_pup/seam_ai/run_2025-02-05-13-23-166cc155570ceb43dbb3c840a389c30f2c.yaml


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | loss_fn | CrossEntropyLoss | 0      | train
1 | model   | _SetR_PUP        | 322 M  | train
-----------------------------------------------------
16.5 M    Trainable params
306 M     Non-trainable params
322 M     Total params
1,291.143 Total estimated model params size (MB)
363       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 2: 100%|██████████| 3/3 [00:01<00:00,  1.95it/s, v_num=m_ai, train_loss_step=2.130, val_loss_step=3.720, val_IoU=0.142, val_loss_epoch=3.720, train_IoU=0.246, train_loss_epoch=2.270] 

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 3/3 [00:57<00:00,  0.05it/s, v_num=m_ai, train_loss_step=2.130, val_loss_step=3.720, val_IoU=0.142, val_loss_epoch=3.720, train_IoU=0.246, train_loss_epoch=2.270]
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/docs/notebooks/examples/seismic/facies_classification/parihaka/logs/setr_pup/seam_ai/run_2025-02-05-13-23-166cc155570ceb43dbb3c840a389c30f2c.yaml


In [8]:
print(f"Checkpoint saved at {pipeline.trainer.checkpoint_callback.last_model_path}")

Checkpoint saved at /workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/docs/notebooks/examples/seismic/facies_classification/parihaka/logs/setr_pup/seam_ai/checkpoints/last.ckpt
